<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://supaerodatascience.github.io/machine-learning/">https://supaerodatascience.github.io/machine-learning/</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Text data pre-processing</div>

In this exercice, we shall load a database of email messages and pre-format them so that we can design automated classification methods or use off-the-shelf classifiers. The general purpose of this notebook is to give a practical notion (through this example) of how important data pre-processing can be in a Machine Learning workflow, and generalize it to other situations.

"What is there to pre-process?" you might ask. Well, actually, text data comes in a very noisy form that we, humans, have become accustomed to and filter out effortlessly to grasp the core meaning of the text. It has a lot of formatting (fonts, colors, typography...), punctuation, abbreviations, common words, grammatical rules, etc. that we might wish to discard before even starting the data analysis.

Here are some pre-processing steps that can be performed on text:
1. loading the data, removing attachements, merging title and body;
2. tokenizing - splitting the text into atomic "words";
3. removal of stop-words - very common words;
4. removal of non-words - punctuation, numbers, gibberish;
3. lemmatization - merge together "find", "finds", "finder".

The final goal is to be able to represent a document as a mathematical object, e.g. a vector, that our machine learning black boxes can process.



<img id="fig1" src="https://imgs.xkcd.com/comics/constructive.png"> 

A tech company comes to you to create a moderation system for their social network : they want to detect spam comments, and later on also detect offensive contents to remove them automatically.


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Load-the-data" data-toc-modified-id="1.-Load-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>1. Load the data</a></span></li><li><span><a href="#2.-Filtering-out-the-noise" data-toc-modified-id="2.-Filtering-out-the-noise-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2. Filtering out the noise</a></span></li><li><span><a href="#3.-Even-better-filtering" data-toc-modified-id="3.-Even-better-filtering-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>3. Even better filtering</a></span></li><li><span><a href="#4.-Term-frequency-times-inverse-document-frequency" data-toc-modified-id="4.-Term-frequency-times-inverse-document-frequency-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>4. Term frequency times inverse document frequency</a></span></li><li><span><a href="#5.-Utility-function" data-toc-modified-id="5.-Utility-function-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>5. Utility function</a></span></li></ul></div>

# 1. Load the data

To showcase our proposed system, we load a database of email messages and pre-format them so that we can design automated classification methods or use off-the-shelf classifiers.


**Questions** :
- What simple statistics could you print on the dataset ?
- Why is there multiple folders on the dataset ?

In [ ]:
#!git clone git@github.com:SupaeroDataScience/machine-learning.git

In [2]:
import os

train_dir = '../data/lingspam_public/bare/'
# train_dir = 'machine-learning/data/ling-spam/train-mails/'

email_path = []
email_label = []
for d in os.listdir(train_dir):
    folder = os.path.join(train_dir,d)
    email_path += [os.path.join(folder,f) for f in os.listdir(folder)]
    email_label += [f[0:3]=='spm' for f in os.listdir(folder)]
print("number of emails",len(email_path))
email_nb = 8 # try 8 for a spam example
print("email file:", email_path[email_nb])
print("email is a spam:", email_label[email_nb])
print(open(email_path[email_nb]).read())

number of emails 2893
email file: ../data/lingspam_public/bare/part6/9-1699msg2.txt
email is a spam: False
Subject: 7th european nlg workshop

7th european workshop on natural language generation call for papers toulouse ( france ) , may 13-15 1999 this seventh european workshop on natural language generation will focus on all aspects of computational linguistics and industrial applications related to natural language generation . papers related to theoretical aspects , applied research and ongoing projects are encouraged . natural language generation being the study of a number of models , approaches and systems , the workshop will include , besides classical topics , scientific domains in which natural language generation plays an important role , such as speech , dialogue , multi-media interfaces , psycho-linguistics , and theoretical linguistics . relevant application domains include all types of applications ( mt , ir and ie , etc . ) in which generation plays an important role an

# 2. Filtering out the noise

One nice thing about scikit-learn is that is has lots of preprocessing utilities. Like [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) for instance, that converts a collection of text documents to a matrix of token counts.

- To remove stop-words, we set: `stop_words='english'`
- To convert all words to lowercase: `lowercase=True`
- The default tokenizer in scikit-learn removes punctuation and only keeps words of more than 2 letters.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
countvect = CountVectorizer(input='filename', stop_words='english', lowercase=True)
word_count = countvect.fit_transform(email_path)

In [4]:
print("Number of documents:", len(email_path))
words = countvect.get_feature_names_out()
print("Number of words:", len(words))
print("Document - words matrix:", word_count.shape)
print("First words:", words[0:100])

Number of documents: 2893
Number of words: 60618
Document - words matrix: (2893, 60618)
First words: ['00' '000' '0000' '00001' '00003000140' '00003003958' '00007' '0001'
 '00010' '00014' '0003' '00036' '000bp' '000s' '000yen' '001' '0010'
 '0010010034' '0011' '00133' '0014' '00170' '0019' '00198' '002' '002656'
 '0027' '003' '0030' '0031' '00333' '0037' '0039' '003n7' '004' '0041'
 '0044' '0049' '005' '0057' '006' '0067' '007' '00710' '0073' '0074'
 '00799' '008' '009' '00919680' '0094' '00a' '00am' '00arrival' '00b'
 '00coffee' '00congress' '00d' '00dinner' '00f' '00h' '00hfstahlke' '00i'
 '00j' '00l' '00m' '00p' '00pm' '00r' '00t' '00tea' '00the' '00uzheb' '01'
 '0100' '01003' '01006' '0104' '0106' '01075' '0108' '011' '0111' '0117'
 '0118' '01202' '01222' '01223' '01225' '01232' '01235' '01273' '013'
 '0131' '01334' '0135' '01364' '0139' '013953' '013a']


# 3. Even better filtering

That's already quite ok, but this pre-processing does not perform lemmatization, the list of stop-words could be better and we could wish to remove non-english words (misspelled, with numbers, etc.).

A slightly better preprocessing uses the [Natural Language Toolkit](https://www.nltk.org/https://www.nltk.org/). The one below:
- tokenizes;
- removes punctuation;
- removes stop-words;
- removes non-English and misspelled words (optional);
- removes 1-character words;
- removes non-alphabetical words (numbers and codes essentially).

In [5]:
# Run only of nltk is not installed
%pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to /home/yann/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/yann/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/yann/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/yann/nltk_data...


True

In [6]:
from nltk import wordpunct_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import words
from string import punctuation
class LemmaTokenizer(object):
    def __init__(self, remove_non_words=True):
        self.wnl = WordNetLemmatizer()
        self.stopwords = set(stopwords.words('english'))
        self.words = set(words.words())
        self.remove_non_words = remove_non_words
    def __call__(self, doc):
        # tokenize words and punctuation
        word_list = wordpunct_tokenize(doc)
        # remove stopwords
        word_list = [word for word in word_list if word not in self.stopwords]
        # remove non words
        if(self.remove_non_words):
            word_list = [word for word in word_list if word in self.words]
        # remove 1-character words
        word_list = [word for word in word_list if len(word)>1]
        # remove non alpha
        word_list = [word for word in word_list if word.isalpha()]
        return [self.wnl.lemmatize(t) for t in word_list]

countvect = CountVectorizer(input='filename',tokenizer=LemmaTokenizer(remove_non_words=True))
word_count = countvect.fit_transform(email_path)
feat2word = {v: k for k, v in countvect.vocabulary_.items()}

/home/yann/py_env/sdd_env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [7]:
print("Number of documents:", len(email_path))
words = countvect.get_feature_names_out()
print("Number of words:", len(words))
print("Document - words matrix:", word_count.shape)
print("First words:", words[0:100])

Number of documents: 2893
Number of words: 14279
Document - words matrix: (2893, 14279)
First words: ['aa' 'aal' 'aba' 'aback' 'abacus' 'abandon' 'abandoned' 'abandonment'
 'abbas' 'abbreviation' 'abdomen' 'abduction' 'abed' 'aberrant'
 'aberration' 'abide' 'abiding' 'abigail' 'ability' 'ablative' 'ablaut'
 'able' 'abler' 'aboard' 'abolition' 'abord' 'aboriginal' 'aborigine'
 'abound' 'abox' 'abreast' 'abridged' 'abroad' 'abrogate' 'abrook'
 'abruptly' 'abscissa' 'absence' 'absent' 'absolute' 'absolutely'
 'absoluteness' 'absolutist' 'absolutive' 'absolutization' 'absorbed'
 'absorption' 'abstract' 'abstraction' 'abstractly' 'abstractness'
 'absurd' 'absurdity' 'abu' 'abundance' 'abundant' 'abuse' 'abusive'
 'abyss' 'academe' 'academic' 'academically' 'academician' 'academy'
 'accelerate' 'accelerated' 'accelerative' 'accent' 'accentuate'
 'accentuation' 'accept' 'acceptability' 'acceptable' 'acceptance'
 'acceptation' 'accepted' 'acception' 'access' 'accessibility'
 'accessible' 'acce

# 4. Term frequency times inverse document frequency

After this first preprocessing, each document is summarized by a vector of size "number of words in the extracted dictionnary". For example, the first email in the list has become:

In [10]:
mail_number = 1
text = open(email_path[mail_number]).read()
print("Original email:")
print(text)
#print(LemmaTokenizer()(text))
#print(len(set(LemmaTokenizer()(text))))
#print(len([feat2word[i] for i in word_count2[mail_number, :].nonzero()[1]]))
#print(len([word_count2[mail_number, i] for i in word_count2[mail_number, :].nonzero()[1]]))
#print(set([feat2word[i] for i in word_count2[mail_number, :].nonzero()[1]])-set(LemmaTokenizer()(text)))
emailBagOfWords = {feat2word[i]: word_count[mail_number, i] for i in word_count[mail_number, :].nonzero()[1]}
print("Bag of words representation (", len(emailBagOfWords), " words in dict):", sep='')
print(emailBagOfWords)
print("\nVector reprensentation (", word_count[mail_number, :].nonzero()[1].shape[0], " non-zero elements):", sep='')
print(word_count[mail_number, :])

Original email:
Subject: theoretical , descriptive , and applied linguistics

call for papers : last announcement kentucky foreign language conference linguistics sessions the 52nd annual kentucky foreign language conference will be held april 22-24 , 1999 , at the university of kentucky in lexington . the conference will include sessions devoted to all aspects of theoretical and descriptive linguistics , sociolinguistics , and applied linguistics . if you wish to present a paper in one of these sessions , send two copies of a one page abstract to prof . anna bosch , 1215 patterson office tower , university of kentucky , lexington , ky 40506-0027 . alternatively , send your abstract by email to : bosch @ pop . uky . edu . ( email submissions are encouraged . ) please include the following information with your abstract : name , affiliation , address , email address , daytime phone . the deadline for submission of abstracts is monday , november 16 , 1998 . authors will be notified about

**Questions** : 
- What is a bag-of-word representation ?
- What kind of feature selection or feature engineering could you derivate from this bag of word representation ?

Counting words is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called `tf` for Term Frequencies.

Another refinement on top of `tf` is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called `tf–idf` for “Term Frequency times Inverse Document Frequency” and again, scikit-learn does the job for us with the [TfidfTransformer](scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) function.

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer().fit_transform(word_count)
tfidf.shape

(2893, 14279)

Now every email in the corpus has a vector representation that filters out unrelevant tokens and retains the significant information.

In [12]:
print("email 0:")
print(tfidf[0,:])

email 0:
  (0, 14243)	0.02575565818614938
  (0, 14188)	0.009037157386654272
  (0, 14184)	0.018914479293583534
  (0, 14165)	0.029251285089143514
  (0, 14129)	0.027374168199134727
  (0, 14128)	0.02347142436978794
  (0, 14095)	0.021020615003509328
  (0, 14000)	0.034397003844170014
  (0, 13952)	0.022218258449752018
  (0, 13907)	0.0651513065186056
  (0, 13818)	0.027708809423839673
  (0, 13807)	0.014715215692291669
  (0, 13799)	0.10909159029227081
  (0, 13758)	0.01477025767369468
  (0, 13741)	0.013254943526520086
  (0, 13699)	0.026618526166468353
  (0, 13664)	0.019505300856334735
  (0, 13613)	0.04002600677129189
  (0, 13427)	0.01395802627701826
  (0, 13310)	0.033543417774697756
  (0, 13190)	0.009744913287464005
  (0, 13170)	0.009722925887796722
  (0, 13141)	0.02300714480492907
  (0, 13140)	0.01748772028398016
  (0, 13114)	0.01839729642565856
  :	:
  (0, 1302)	0.05546000340672882
  (0, 1257)	0.012922317342736063
  (0, 1085)	0.02631056242208824
  (0, 1084)	0.1078046094010095
  (0, 1081)	0.1021

# 5. Utility function

Let's put all this loading process into a separate file so that we can reuse it in other experiments.

In [13]:
import load_spam
spam_data = load_spam.spam_data_loader()
spam_data.load_data()

In [14]:
spam_data.print_email(8)

email file: ../data/lingspam_public/bare/part6/9-1699msg2.txt
email is a spam: False
Subject: 7th european nlg workshop

7th european workshop on natural language generation call for papers toulouse ( france ) , may 13-15 1999 this seventh european workshop on natural language generation will focus on all aspects of computational linguistics and industrial applications related to natural language generation . papers related to theoretical aspects , applied research and ongoing projects are encouraged . natural language generation being the study of a number of models , approaches and systems , the workshop will include , besides classical topics , scientific domains in which natural language generation plays an important role , such as speech , dialogue , multi-media interfaces , psycho-linguistics , and theoretical linguistics . relevant application domains include all types of applications ( mt , ir and ie , etc . ) in which generation plays an important role and where specific techn